In [4]:
#cudf for linux
%conda install -c rapidsai -c conda-forge -c nvidia cudf=23.04 python=3.10 cudatoolkit=11.8
#https://github.com/rapidsai/cudf

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - cudf=23.04

Current channels:

  - https://conda.anaconda.org/rapidsai/win-64
  - https://conda.anaconda.org/rapidsai/noarch
  - https://conda.anaconda.org/conda-forge/win-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://conda.anaconda.org/nvidia/win-64
  - https://conda.anaconda.org/nvidia/noarch
  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [5]:
import pandas as pd
# import cudf

# Load data to df
## Load data file

In [6]:
import os
from os.path import join

#Gowalla setting
#================================================
DATA_NAME = "Gowalla"
FILE_NAME = "Gowalla_totalCheckins.txt"
COL_NAMES = ['user','time','lat','long','item']
#================================================

# config
ROOT_PATH = os.path.abspath("")
DATA_PATH = join(ROOT_PATH+"\\data", DATA_NAME)
FILE_PATH = join(DATA_PATH, FILE_NAME)
FILT_THRESHOLD = 10


In [7]:
# Load File
origin_df = pd.read_table(FILE_PATH, names=COL_NAMES)
print(f"Total Edges : {len(origin_df)}\nTotal User : {len(origin_df['user'].unique())}\nTotal item : {len(origin_df['item'].unique())} \
        \n Smallest user id : {origin_df['user'].unique().min()} \
        \n biggest user id : {origin_df['user'].unique().max()} \
        \n Smallest item id : {origin_df['item'].unique().min()} \
        \n biggest item id : {origin_df['item'].unique().max()} \
        \n Min Interaction Per user : {origin_df.user.value_counts().min()} \
        \n Max Interaction Per user : {origin_df.user.value_counts().max()} \
        \n Avg Interaction Per user : {origin_df.user.value_counts().mean()}\
        \n Min Interaction Per item : {origin_df.item.value_counts().min()} \
        \n Max Interaction Per item : {origin_df.item.value_counts().max()} \
        \n Avg Interaction Per item : {origin_df.item.value_counts().mean()}")

Total Edges : 6442892
Total User : 107092
Total item : 1280969         
 Smallest user id : 0         
 biggest user id : 196585         
 Smallest item id : 8904         
 biggest item id : 5977757         
 Min Interaction Per user : 1         
 Max Interaction Per user : 2175         
 Avg Interaction Per user : 60.16221566503567        
 Min Interaction Per item : 1         
 Max Interaction Per item : 5811         
 Avg Interaction Per item : 5.029701733609478


In [8]:
from tqdm import tqdm
# Change each user and item index start from 0
origin_df.sort_values(by='user',ascending=True)

# Change user id to start from 0
id = 0

for user_id in tqdm(origin_df['user'].unique()):
    origin_df = origin_df.replace({'user':{user_id:id}})
    id += 1

# Change item id to start from 0
origin_df.sort_values(by='item',ascending=True)
id = 0

for item_id in tqdm(origin_df['item'].unique()):
    origin_df = origin_df.replace({'item':{item_id:id}})
    id += 1

origin_df

  0%|          | 204/107092 [00:36<5:24:19,  5.49it/s]

In [ ]:
# Save refactored dataframe
origin_df.to_csv(DATA_NAME+"_refactored.txt",index=False,header=None)

In [ ]:
# Load refactored dataframe
origin_df = pd.read_table(FILE_PATH + DATA_NAME+"_refactored.txt", names=COL_NAMES)

## User-Item Train-Test Split

In [4]:
df = origin_df
# Just interaction and drop duplicate
df = df[['user','item']]
df = df.drop_duplicates()

In [5]:
# Save Interaction information without filtering
WO_filtering = df
# Sort with ascending order with user first and item
WO_filtering = WO_filtering.sort_values(by=['user','item'], ascending=[True,True])

In [6]:
WO_filtering_filename = DATA_NAME + "_WO_Filtering.intr"
if not os.path.exists(WO_filtering_filename):
    WO_filtering.to_csv(WO_filtering_filename,sep=" ", index=False, header=None)

In [9]:
filtering_filename = DATA_NAME + "Filtering.intr"
df_usr = df.groupby('user').count()
df_item = df.groupby('item').count()
print(df_item)

         user
item         
8904        7
8932       13
8936        7
8938       28
8947      268
...       ...
5975123     1
5976149     1
5976173     1
5977211     1
5977757     1

[1280969 rows x 1 columns]


## User-Item Incremental Split